In [23]:
import cv2 as cv
import numpy as np

In [315]:
cap = cv.VideoCapture('dataset\\videos\\video08.mp4')

In [316]:
mog2 = cv.createBackgroundSubtractorMOG2(varThreshold=100, detectShadows=False)

In [317]:
while(True):
    ret, frame = cap.read()

    if not ret: 
        break
    
    fgmask = mog2.apply(frame)

    fgmask = cv.erode(fgmask, np.ones((5,5), np.uint8))
    fgmask = cv.dilate(fgmask, np.ones((41,41), np.uint8))
    
    fgmask = cv.morphologyEx(fgmask, cv.MORPH_CLOSE, np.ones((45, 45), np.uint8))

    _, binary_mask = cv.threshold(fgmask, 1, 255, cv.THRESH_BINARY)

    contours, _ = cv.findContours(binary_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    contour_sort = sorted(contours, key=cv.contourArea, reverse=True)

    for i, contour in enumerate(contours):
        x, y, w, h = cv.boundingRect(contour)

        area = cv.contourArea(contour)

        detected_width_range = 50 < (x + w) - x < 1000
        detected_height_range = (100 < (y + h) - y < 700)
        if (area >= 6000 and area <= 150000) and detected_height_range and detected_height_range:
            cv.rectangle(frame, (x,y), (x + w, y + h), (0,0,255), 2)

            cv.putText(frame, f"Carro", (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv.imshow('frame', frame)

    if cv.waitKey(60) & 0xff == 27:
        break

cap.release()
cv.destroyAllWindows()